# Make Echo transfer files for validation 1D plates (exp101)

In [ ]:
import csv
import pathlib
import sys
from collections import Counter

sys.path.append(str(pathlib.Path("__file__").absolute().parents[2]))
from labware.transfers import Transfer, TransferStep
from labware.plates import Plate384

In [ ]:
header = ['Source Barcode', 'Source Well', 'Destination Barcode', 'Destination Well', 'Volume']
header

In [ ]:
# static things
# n.b. Source1 / Synthesis1 are used in concurrent exp100, for that reason we start at 2
source_barcode = 'Source2'
destination_barcode = ['Synthesis2', 'Synthesis3', 'Synthesis4']
volume = [1100, 990]

# define source wells
initiator_wells = [
    ['A1', 'A2'], 
    ['A3', 'A4'], 
    ['A5', 'A6'], 
    ['A7', 'A8'], 
    ['A9', 'A10'], 
    ['A11', 'A12'], 
    ['A13', 'A14'], 
    ['A15', 'A16'], 
    ['A17', 'A18'], 
    ['A19', 'A20'], 
    ['A21', 'A22'], 
    ['A23', 'A24'], 
]

monomer_wells = [
    ['F1', 'F2'], 
    ['F3', 'F4'], 
    ['F5', 'F6'], 
    ['F7', 'F8'], 
    ['F9', 'F10'], 
    ['F11', 'F12'], 
    ['F13', 'F14'], 
    ['F15', 'F16'], 
    ['F17', 'F18'], 
    ['F19', 'F20'], 
]

terminator_wells = [
    ['K1', 'K2', 'K3'],
    ['K4', 'K5', 'K6'],
    ['K7', 'K8', 'K9'],
    ['K10', 'K11', 'K12'],
    ['K13', 'K14', 'K15'],
    ['K16', 'K17', 'K18'],
    ['K19', 'K20', 'K21'],
    ['K22', 'K23', 'K24'],
    ]

# for destination wells list, index 0 is the upper left half of the plate, 1 the upper right, 2 the lower left and 3 the lower right
# n.b. this will fill 320 wells in a 384 well plate
initiator_destination_wells = [
    [f'{chr(i // 10 + 65)}{i % 10 + 3}' for i in range(80)], 
    [f'{chr(i // 10 + 65)}{i % 10 + 13}' for i in range(80)], 
    [f'{chr(i // 10 + 73)}{i % 10 + 3}' for i in range(80)], 
    [f'{chr(i // 10 + 73)}{i % 10 + 13}' for i in range(80)]
]

monomer_destination_wells = [[f"{chr(s+65)}{i}" for s in range(16)] for i in range(3, 23)]

terminator_destination_wells = [[f"{chr(s+65)}{i}" for i in range(3, 23)] for s in range(16)]


In [ ]:
# some checks"
assert 3 * len(initiator_destination_wells) == len(initiator_wells)
assert len(monomer_destination_wells) == 2 * len(monomer_wells)
assert len(terminator_destination_wells) == 2 * len(terminator_wells)

In [ ]:
lines = []
lines.insert(0, header)
lines

In [ ]:
# fill 12 initiators, in four quadrants of three plates
vol = 990
for i, dest_code in enumerate(destination_barcode):
    for ini_wells, dest_wells in zip(initiator_wells[i*4:i*4+4], initiator_destination_wells):
        for dest_well in dest_wells[:len(dest_wells) // 2]:
            source_well = ini_wells[0]
            lines.append([source_barcode, source_well, dest_code, dest_well, vol])
        for dest_well in dest_wells[len(dest_wells) // 2:]:
            source_well = ini_wells[1]
            lines.append([source_barcode, source_well, dest_code, dest_well, vol])

In [ ]:
# fill 10 monomers in 10 columns, twice for each plate, for three plates
vol = 990
for dest_code in destination_barcode:
    for source_wells, dest_wells in zip(monomer_wells, monomer_destination_wells[:len(monomer_destination_wells) // 2]): 
        for dest_well in dest_wells:
            lines.append([source_barcode, source_wells[0], dest_code, dest_well, vol])
    for source_wells, dest_wells in zip(monomer_wells, monomer_destination_wells[len(monomer_destination_wells) // 2:]): 
        for dest_well in dest_wells:
            lines.append([source_barcode, source_wells[1], dest_code, dest_well, vol])

In [ ]:
# fill oxalic acid in all wells. One source well of oxalic acid is used for half a plate (A3-P12 / A13-P22)
vol = 220
for dest_code, source_wells in zip(destination_barcode, [["P1", "P2"], ["P3", "P4"], ["P5", "P6"]]):
    for dest_well in [f"{chr(s+65)}{i}" for s in range(16) for i in range(3, 13)]:
        lines.append([source_barcode, source_wells[0], dest_code, dest_well, vol])
    for dest_well in [f"{chr(s+65)}{i}" for s in range(16) for i in range(13, 23)]:
        lines.append([source_barcode, source_wells[1], dest_code, dest_well, vol])


In [ ]:
len(lines)

At this point, I/M transfers are finished. We write some tests before saving.

In [ ]:
# correct number of transfers?
assert len(lines) == 80 * 4 * 3 * 3 + 1
# initiator source wells occur 40 times, monomer source wells 48 times?
used_wells = [l[1] for l in lines[1:]]
for k, v in Counter(used_wells).items():
    if k.startswith("A"):
        assert v == 40
    elif k.startswith("F"):
        assert v == 48
    elif k.startswith("P"):
        assert v == 160
    else:
        raise ValueError(f"unexpected well {k}")
# all transfers are unique
assert len(lines) == len(set([tuple(line) for line in lines]))
# all destination wells are used exactly thrice
used_dest_wells = [l[2] + "_" + l[3] for l in lines[1:]]
for k, v in Counter(used_dest_wells).items():
    assert v == 3

In [ ]:
# save to file
with open('validation_exp101_step1.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(lines)

Let's move on to T transfers.

In [ ]:
lines = []
lines.insert(0, header)
lines

In [ ]:
# fill 8 monomers in 8 rows, twice for each plate, for three plates
vol = 1100
for dest_code in destination_barcode:
    for source_wells, dest_wells in zip(terminator_wells, terminator_destination_wells[:len(terminator_destination_wells) // 2]): 
        for dest_well in dest_wells:
            lines.append([source_barcode, source_wells[0], dest_code, dest_well, vol])
    for source_wells, dest_wells in zip(terminator_wells, terminator_destination_wells[len(terminator_destination_wells) // 2:]): 
        for dest_well in dest_wells:
            lines.append([source_barcode, source_wells[1], dest_code, dest_well, vol])

In [ ]:
len(lines)

At this point, T transfers are finished. We write some tests before saving.

In [ ]:
# correct number of transfers?
assert len(lines) == 320 * 3 + 1
# terminator source wells occur 60 times?
used_wells = [l[1] for l in lines[1:]]
for v in Counter(used_wells).values():
    assert v == 60
# all transfers are unique
assert len(lines) == len(set([tuple(line) for line in lines]))
# all destination wells are used exactly once
used_dest_wells = [l[2] + "_" + l[3] for l in lines[1:]]
for k, v in Counter(used_dest_wells).items():
    assert v == 1

In [ ]:
# save to file
with open('validation_exp101_step2.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(lines)


In [ ]:
# load all transfers
transfers = []
with open('validation_exp101_step1.csv', 'r') as file:
    reader = csv.reader(file)
    transfers.extend([line for i, line in enumerate(reader) if i > 0])
with open('validation_exp101_step2.csv', 'r') as file:
    reader = csv.reader(file)
    transfers.extend([line for i, line in enumerate(reader) if i > 0])
transfer = Transfer(transfer_steps=[TransferStep(*t) for t in transfers])

In [ ]:
transfers

In [ ]:
transfer

In [ ]:
# load the source plate
# n.b. we are not using the real volumes here because a mistake was made in the actual setup of the plate 
# and some wells ran out of fluid during the dispense.
source = Plate384(max_vol=165000, dead_vol=15000)
source.from_csv("../../data/plates/exp101/source_plate_layout.csv", vol=165000)
print(source)

In [ ]:
destination_plates = transfer.simulate(source_plates={"Source2": source}, destination_well_number=384)

In [ ]:
# remove the "X" for oxalic acid before saving
for plate in destination_plates.values():
    for well in plate.wells():
        try:
            plate.compounds(well).remove("X")
        except ValueError:
            pass


In [ ]:
for k, plate in destination_plates.items():
    plate.to_csv(f"../../data/plates/exp101/plate_layout_{k}.csv", save_volumes=True)